In [17]:
# the first thing is to import pandas and datatime
import matplotlib.pyplot as plt
import csv
import shapefile
import pandas as pd
from shapely.geometry import Point, Polygon, mapping
from fiona import collection
import geopandas as gpd
import shapely.geometry as Polygon
import datetime
from datetime import datetime
import numpy as np
import filecmp
from geopandas import GeoDataFrame as gdf
import cartopy.crs as ccrs
import cartopy.feature as cfeature
#MAGIC  - render the figure in the notebook.
%matplotlib inline

# we know that the lat long is north-west, turn them into floating pt values
def lat_lon_to_float(v):  
    '''convert strings from NHC to float locations'''
    if(v[-1] == 'S') or (v[-1] == 'W'):
        multiplier = -1
    else:
        multiplier = 1
    return float(v[:-1]) * multiplier

hurdata = []
with open ('hurdat2.txt','r') as f:
    for line in f.readlines():
        if line.startswith('AL'):
            storm_id = line.split(',')
            storm_number = storm_id[0].strip()
            storm_name = storm_id[1].strip() 
        else:
            location_line = line.split(',')
            dt = datetime.strptime(location_line[0]+location_line[1], '%Y%m%d %H%M' )
            storm_status = location_line[3].strip()
            storm_lat = lat_lon_to_float(location_line[4].strip())
            storm_lon = lat_lon_to_float(location_line[5].strip())
            max_speed = float(location_line[7].strip())
            hurdata.append([storm_number, storm_name, storm_status, storm_lat, storm_lon, dt, max_speed])

df = pd.DataFrame(hurdata, columns= ['Storm Number','Storm Name','Storm Status','Lat','Lon','Time', 'Max Speed'])
df.head()

,Storm Number,Storm Name,Storm Status,Lat,Lon,Time,Max Speed
0,AL011851,UNNAMED,HU,28.0,-94.8,1851-06-25 00:00:00,-999.0
1,AL011851,UNNAMED,HU,28.0,-95.4,1851-06-25 06:00:00,-999.0
2,AL011851,UNNAMED,HU,28.0,-96.0,1851-06-25 12:00:00,-999.0
3,AL011851,UNNAMED,HU,28.1,-96.5,1851-06-25 18:00:00,-999.0
4,AL011851,UNNAMED,HU,28.2,-96.8,1851-06-25 21:00:00,-999.0


In [69]:
df1 = df[df['Storm Name'] == "FLORENCE"]# this is a simple boolean mask
df1

,Storm Number,Storm Name,Storm Status,Lat,Lon,Time,Max Speed
23392,AL091953,FLORENCE,TS,15.0,-78.0,1953-09-23 12:00:00,-999.0
23393,AL091953,FLORENCE,TS,16.3,-79.5,1953-09-23 18:00:00,-999.0
23394,AL091953,FLORENCE,TS,17.6,-81.0,1953-09-24 00:00:00,-999.0
23395,AL091953,FLORENCE,TS,18.9,-82.5,1953-09-24 06:00:00,-999.0
23396,AL091953,FLORENCE,TS,20.2,-84.0,1953-09-24 12:00:00,-999.0
...,...,...,...,...,...,...,...
50968,AL062018,FLORENCE,EX,37.8,-82.0,2018-09-17 12:00:00,1008.0
50969,AL062018,FLORENCE,EX,38.8,-82.0,2018-09-17 18:00:00,1008.0
50970,AL062018,FLORENCE,EX,39.5,-80.5,2018-09-18 00:00:00,1008.0
50971,AL062018,FLORENCE,EX,41.3,-76.8,2018-09-18 06:00:00,1007.0


In [85]:
#another filter Criteria
filter1 = (df["Storm Name"] =="FLORENCE")&(df["Lon"]<= 75.0)
filter1

0        False
1        False
2        False
3        False
4        False
         ...  
51812    False
51813    False
51814    False
51815    False
51816    False
Length: 51817, dtype: bool